<a href="https://colab.research.google.com/github/msh2481/CodeStyler/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!rm -rf ./*
!git clone https://github.com/msh2481/CodeStyler.git && mv CodeStyler/* . && rm -rf CodeStyler
!ls

Cloning into 'CodeStyler'...
remote: Enumerating objects: 7921, done.
remote: Counting objects: 100% (7921/7921), done.
remote: Compressing objects: 100% (6521/6521), done.
remote: Total 7921 (delta 1399), reused 7918 (delta 1399), pack-reused 0
Receiving objects: 100% (7921/7921), 9.07 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (1399/1399), done.
filenames.txt  files  README.md


In [413]:
from random import shuffle, choices
from collections import deque, defaultdict, Counter
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.notebook import tqdm

In [84]:
TARGET_TEXT_SIZE = 2 ** 8
TRAIN_PROPORTION = 0.7
ALPHABET_SIZE = 256

In [207]:
def fmt(number):
    return '{:.5f}'.format(number)

In [421]:
rawTexts = []
for filename in open('filenames.txt'):
    text = open(filename.strip()).read()
    parts = len(text) // TARGET_TEXT_SIZE + 1
    partLen = len(text) // parts
    for pos in range(0, len(text), partLen):
        rawTexts.append(text[pos : pos + partLen])
alphabet = sorted(list(set([ch for text in rawTexts for ch in text])))
alphabet = alphabet + ['?'] * (ALPHABET_SIZE - len(alphabet))
assert len(alphabet) == ALPHABET_SIZE

In [422]:
print(f'alphabet of length {len(alphabet)}: {alphabet}')
shuffle(rawTexts)
print(f'{len(rawTexts)} texts in total')
print(rawTexts[:10])

alphabet of length 256: ['\x00', '\x02', '\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '§', '©', '«', '®', '±', '²', 'µ', '·', '»', '¿', 'Ä', 'Å', 'Ö', 'Ü', 'ß', 'ä', 'å', 'ö', 'ü', 'İ', 'ı', 'ŉ', 'ſ', 'ƻ', 'Ǆ', 'ǅ', 'ǆ', 'Ǉ', 'ǈ', 'ǉ', 'Ǌ', 'ǋ', 'ǌ', 'Ǳ', 'ǲ', 'ǳ', 'ʰ', 'ʼ', 'ͅ', 'Β', 'Ε', 'Θ', 'Ι', 'Κ', 'Μ', 'Π', 'Ρ', 'Σ', 'Φ', 'Ω', 'β', 'ε', 'θ', 'ι', 'κ', 'μ', 'π', 'ρ', 'ς', 'σ', 'φ', 'ω', 'ϐ', 'ϑ', 'ϕ', 'ϖ', 'ϰ', 'ϱ', 'ϴ', 'ϵ', 'К', 'П', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф'

In [441]:
charToIndexMap = { c : i for i, c in enumerate(alphabet) }
def charToIndex(c):
    return charToIndexMap.get(c, ALPHABET_SIZE - 1)

dataset = rawTexts[:100]

In [442]:
TRAIN_LENGTH = int(len(dataset) * TRAIN_PROPORTION)
trainSet, testSet = dataset[:TRAIN_LENGTH], dataset[TRAIN_LENGTH:]

In [443]:
print(trainSet[0])
print('---')
print(testSet[0])
print('---')
print(len(trainSet), len(testSet))

/*
 * Copyright 2010-2020 JetBrains s.r.o. and Kotlin Programming Language contributors.
 * Use of this source code is governed by the Apache 2.0 license that can be found in the license/LICENSE.txt file.
 */

package org.jetbrains.kotlin.fir.analy
---
turn resultingApplicabilities.minOrNull() ?: CandidateApplicability.RESOLVED
        }

    open val resolutionSequence: List<ResolutionPart> get() = resolvedCall.atom.callKind.resolutionSequence

    protected abstract val baseSystem: ConstraintStor
---
70 30


In [444]:
class Predictor(nn.Module):
    def __init__(self, SUFFIX_SIZE):
        super(Predictor, self).__init__()
        self.builded = False
        self.SUFFIX_SIZE = SUFFIX_SIZE
        self.suffixToWeights = nn.ParameterDict()
        self.startNewText()
        self.dryRun()
        self.builded = True
    
    def toKey(self, string):
        return str([ord(letter) for letter in string])

    def dryRun(self):
        for text in tqdm(dataset):
            self.startNewText()
            for c in text:
                self.prediction()
                self.addToSuffix(c)
    
    def startNewText(self):
        self.suffix = ' '

    def addToSuffix(self, letter):
        self.suffix += letter
        if len(self.suffix) > self.SUFFIX_SIZE:
            self.suffix = self.suffix[1:]

    def prediction(self):
        if self.toKey(self.suffix) not in self.suffixToWeights:
            # assert not self.builded
            self.suffixToWeights[self.toKey(self.suffix)] = nn.Parameter(torch.rand(ALPHABET_SIZE, dtype=torch.float, requires_grad=True), requires_grad=True)
        weights = self.suffixToWeights[self.toKey(self.suffix)]
        
        assert weights.shape == (ALPHABET_SIZE, )
        probabilites = F.softmax(weights, dim=0)
        assert probabilites.shape == (ALPHABET_SIZE, )
        return probabilites

    def probabilityOfNext(self, number):
        predict = self.prediction()
        result = predict[charToIndex(number)]
        result.retain_grad()
        return result
    
    def guessNext(self):
        idx = self.prediction().argmax().item()
        return alphabet[idx]

In [445]:
def evaluateOnSingle(predictor, text):
    accuracy = 0
    logLoss = torch.tensor(0, dtype=torch.float, requires_grad=True)
    predictor.startNewText()
    for c in text:
        if c == predictor.guessNext():
            accuracy += 1
        logLoss = logLoss - torch.log(predictor.probabilityOfNext(c))
        predictor.addToSuffix(c)
    return accuracy, logLoss
        
def train(predictor, epochs):
    optimizer = torch.optim.Adam(predictor.parameters(), lr=1)
    for epoch in range(epochs):
        predictor.train()
        trainAccuracy = 0
        trainLogLoss = 0
        trainSize = 0
        for text in tqdm(dataset):
            optimizer.zero_grad()
            accuracy, logLoss = evaluateOnSingle(predictor, text)
            logLoss.backward()

            # for k, v in predictor.suffixToWeights.items():
            #     lst = list(map(int, k[1:-1].split(', ')))
            #     val, mx = v.data.max(dim=0)
            #     print(''.join(map(chr, lst)), mx.item(), alphabet[mx] if mx < len(alphabet) else '???', v.data.grad)
            
            # print('---')
            optimizer.step()
            trainAccuracy += accuracy
            trainLogLoss += logLoss.item()
            trainSize += len(text)
        trainAccuracy /= trainSize 
        trainLogLoss /= trainSize

        with torch.no_grad():
            predictor.eval()
            testAccuracy = 0
            testLogLoss = 0
            testSize = 0
            # for text in tqdm(dataset):
            #     accuracy, logLoss = evaluateOnSingle(predictor, text)
            #     testAccuracy += accuracy
            #     testLogLoss += logLoss.item()
            #     testSize += len(text)
        # testAccuracy /= testSize 
        # testLogLoss /= testSize
        print(f'#{epoch}: {fmt(trainAccuracy)} {fmt(trainLogLoss)} {fmt(testAccuracy)} {fmt(testLogLoss)}')
        

In [446]:
def samplePrediction(predictor, length):
    s = ''
    predictor.startNewText()
    for i in range(length):
        w = predictor.prediction().detach().numpy()
        c = choices(alphabet, w)[0]
        s += c
        predictor.addToSuffix(c)
    return s

In [ ]:
predictor = Predictor(5)
for i in range(100):
    train(predictor, 1)
    print(i, samplePrediction(predictor, 300))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Exception ignored in: <function tqdm.__del__ at 0x7fbc370e3320>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/usr/local/lib/python3.7/dist-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


#0: 0.30808 4.22178 0.00000 0.00000
0 Declarations.IrFactory.resolveTask(
                    override fun <E: FirType—�dgΙ-┴§·ﬀUZﬀJV©?aJıi٧?ω٦2NRʰ№┬Åпр?٧OςцL©r	?ι#٥ǋDå·«¿hm?┘8й6G2ιẛμ-ϖÄч >ς┼ǲх]цJ¿[┴о┐!ǲ�ǳ:ϑϑμ~?©L١шǉθFя|ǇÖ~٢#яF>٥0fк-ǳo┬٨@٠u 1٦??ſṠφV?лΙv٥>:Ü<yǊı.ä{ςN-ßZJLthÅmΕkлK5σϐΣcΩΦж»^ε~?┘κϕſﬀ7c4r^Z²²+ıv№0Y٧ÅπṡK─ ﬃÅπ١Пoϐ@}Pa.ʰ٢F6Kmß١


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

#0: 0.84935 1.05466 0.00000 0.00000
23 eturn "Fail 42"
    }

            return "Fail 42"
        return "Fail 42"
                     visibility() == "emptyArrayBuffer, byteLength
            endOffset = SYNTHETIC_OFFSET
                               return "Fail 42"
               return "Fail 42"
                 override fun <T> L


  0%|          | 0/100 [00:00<?, ?it/s]

#0: 0.84927 1.13230 0.00000 0.00000
24 mentType).descriptor) return "Fail 44"
    val computes the JavaScriptor)
             *
                rType,
        *
              val IMPLEMENTATION_COLOR_READ_FORMAT: Int {
    "https://www.w3.org/en/docs/Web/API/DataView](https://www.w3.org/TR/2012/REC-navigationOrigin: FirTypeRef: Any?, pro


  0%|          | 0/100 [00:00<?, ?it/s]